# 단어 Level로 번역기 업그레이드하기

In [2]:
# 필요한 모듈 import
import os
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

## Step 1. 정제, 정규화, 전처리

In [39]:
# 파일 불러오기
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 197463


,eng,fra,cc
14459,I saw him do it.,Je l'ai vu le faire.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
135081,How long do you plan to stay here?,Combien de temps prévoyez-vous de rester ici ?,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
10353,I did warn you.,Je vous avais prévenue.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
101016,Do you want some mulled wine?,Veux-tu du vin chaud ?,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
136729,"I'm sorry, I didn't recognize you.","Je suis désolé, je ne vous ai pas reconnu.",CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [48]:
# 컬럼 cc제거 및 3만3천개 샘플 사용
lines = lines[['eng', 'fra']][:33000]
lines.sample(5)

,eng,fra
26776,Tom looks curious.,Tom semble curieux.
8144,It's very big.,C'est très grand.
30580,I'm from Singapore.,Je suis de Singapour.
6475,You go first.,Vous en premier.
1215,I'm sorry.,Excusez-moi.


In [44]:
# 구두점(Punctuation)을 단어와 분리
import re
lines.eng = re.sub(r"([^\w])", r" \1 ", lines.eng) # 구두점 양쪽에 공백 넣기
lines.eng = lines.eng.split( ) # 공백 기준으로 분리
lines.sample(5)

TypeError: expected string or bytes-like object

- 구두점을 단어와 분리하기 위해서 처음에 특수문자를 문자열화해서 자르려고 시도했지만 당연히 안됐다. 여러 방법을 찾아보다가 주완님과 상규님의 도움을 받아서 해결했다. 그래서 인공작사가 만들기 문장 전처리를 참고해서 특수문자 양쪽에 공백을 넣고 그 공백을 기준으로 split하면 된다는 생각이 들었고, 그렇게 해결했다.

In [46]:
import re
string = 'he is a good boy!'
string = re.sub(r"([^\w])", r" \1 ", string) # 특수문자 양쪽에 공백 넣기
string.split( )

['he', 'is', 'a', 'good', 'boy', '!']

In [50]:
lines = lines['eng'].str.lower()
lines = lines['fra'].str.lower()
lines.sample(5)

KeyError: 'fra'

## Step 2. 디코더의 문장에 시작 토큰과 종료 토큰을 넣어주기

In [ ]:
# 시작 토큰과 종료 토큰 추가
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t ' + x + ' \n')
# fra 각 열에 앞 뒤로 각각 \t와 \n을 넣기 

print('전체 샘플의 수 :',len(lines))
lines.sample(5)

## Step 3. 케라스의 토크나이저로 텍스트를 숫자로 바꾸기